# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [6]:
# trusted aggregator : a neutral 3rd party who has a machine that we can trust to not look at the gradient when performing the aggregation.
# rather than depennding on the db owner who might be inlined to other data partners , we can choose a 3rd neutral party ...anyone in the planet to larger pool to find the trust worthy person
# send all their grad to neutarl 3rd party 

import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0807 19:29:58.945861 14556 hook.py:98] Torch was already hooked... skipping hooking process


In [7]:
# create workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [8]:
# connecting them and inform others of their exixtance
# needed in real world not needed in virtual setup

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([bob, alice])

W0807 19:30:04.297513 14556 base.py:646] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0807 19:30:04.300499 14556 base.py:646] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0807 19:30:04.304496 14556 base.py:646] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0807 19:30:04.308484 14556 base.py:646] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0807 19:30:04.312476 14556 base.py:646] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0807 19:30:04.316464 14556 base.py:646] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [9]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

In [10]:
# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

In [11]:
alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [12]:
# create our model 
# Iniitalize A Toy Model
model = nn.Linear(2,1)

In [13]:
# Send a Copy of The Model to Alice and Bob¶
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

In [14]:
# Train Bob's and Alice's Models (in parallel)
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    alices_loss

In [15]:
# Send Both Updated Models to a Secure Worker
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

In [16]:
# Average The Models
with th.no_grad():

    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

In [17]:
#  Rinse and Repeat
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    with th.no_grad():

        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0022) Alice:tensor(0.0014)
Bob:tensor(0.0011) Alice:tensor(0.0004)
Bob:tensor(0.0006) Alice:tensor(0.0001)
Bob:tensor(0.0004) Alice:tensor(5.5874e-05)
Bob:tensor(0.0002) Alice:tensor(2.0924e-05)
Bob:tensor(0.0001) Alice:tensor(7.3734e-06)
Bob:tensor(9.2735e-05) Alice:tensor(2.3406e-06)
Bob:tensor(6.2764e-05) Alice:tensor(6.1129e-07)
Bob:tensor(4.3626e-05) Alice:tensor(1.0082e-07)
Bob:tensor(3.1001e-05) Alice:tensor(1.3230e-09)


In [18]:
preds = model(data)
loss = ((preds - target) ** 2).sum()

In [19]:
print(preds)
print(target)
print(loss.data)

tensor([[0.0141],
        [0.0107],
        [0.9867],
        [0.9833]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0008)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [20]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [21]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [22]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [23]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [24]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [25]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [26]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [28]:
import random

Q = 23740629843760239486723

def encrypt(x, n_share=3):
    
    shares = list()
    
    for i in range(n_share-1):
        shares.append(random.randint(0,Q))
        
    shares.append(Q - (sum(shares) % Q) + x)
    
    return tuple(shares)

def decrypt(shares):
    return sum(shares) % Q

In [29]:
shares = encrypt(3)
shares

(8414002419785574960425, 891093902563858252780, 14435533521410806273521)

In [30]:
decrypt(shares)

3

In [31]:
def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [32]:
x = encrypt(5)
y = encrypt(7)
z = add(x,y)
decrypt(z)

12

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [34]:
BASE=10 # base 10 encoding
PRECISION=4
Q = 56354382876297897

In [35]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [36]:
encode(3.5)

35000

In [37]:
decode(35000)

3.5

In [38]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [39]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [40]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [41]:
x = x.share(bob, alice, secure_worker)

In [42]:
bob._objects

{41202046477: tensor([1865942481599103878,  477085160373647769, 4117963359367390229,
         4300543209468077350, 2740599662690764576])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [43]:
y = x + x

In [44]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:45804914245 -> bob:56970803418]
	-> [PointerTensor | me:72603068999 -> alice:55543988757]
	-> [PointerTensor | me:70914161736 -> secure_worker:36016534355]
	*crypto provider: me*

In [45]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [46]:
x = th.tensor([0.1,0.2,0.3])

In [47]:
x

tensor([0.1000, 0.2000, 0.3000])

In [48]:
x = x.fix_prec()

In [49]:
x.child.child

tensor([100, 200, 300])

In [50]:
y = x + x

In [51]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [52]:
x = th.tensor([0.1, 0.2, 0.3])

In [53]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [54]:
y = x + x

In [55]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation